In [62]:
import numpy as np
import hashlib

class ABSplitter:
    def __init__(self, count_slots, salt_one, salt_two):
        self.count_slots = count_slots
        self.salt_one = salt_one
        self.salt_two = salt_two

        self.slots = np.arange(count_slots)
        self.experiments = []
        self.experiment_to_slots = dict()
        self.slot_to_experiments = dict()

    def split_experiments(self, experiments):
        """Устанавливает множество экспериментов, распределяет их по слотам.

        Нужно определить атрибуты класса:
            self.experiments - список словарей с экспериментами
            self.experiment_to_slots - словарь, {эксперимент: слоты}
            self.slot_to_experiments - словарь, {слот: эксперименты}
        experiments - список словарей, описывающих пилот. Словари содержит три ключа:
            experiment_id - идентификатор пилота,
            count_slots - необходимое кол-во слотов,
            conflict_experiments - list, идентификаторы несовместных экспериментов.
            Пример: {'experiment_id': 'exp_16', 'count_slots': 3, 'conflict_experiments': ['exp_13']}
        return: List[dict], список экспериментов, которые не удалось разместить по слотам.
            Возвращает пустой список, если всем экспериментам хватило слотов.
        """
        self.experiments = experiments
        experiments = sorted(experiments, key=lambda x: len(x['conflict_experiments']), reverse=True)

        self.slot_to_experiments = {slot: [] for slot in self.slots}
        self.experiment_to_slots = {experiment['experiment_id']: [] for experiment in experiments}
        unassigned_experiments = []
        for experiment in experiments:
            if experiment['count_slots'] > len(self.slots):
                print(f'ERROR: experiment_id={experiment["experiment_id"]} needs too many slots.')
                unassigned_experiments.append(experiment)
                continue

            # найдём доступные слоты
            notavailable_slots = []
            for conflict_experiment_id in experiment['conflict_experiments']:
                notavailable_slots += self.experiment_to_slots[conflict_experiment_id]
            available_slots = list(set(self.slots) - set(notavailable_slots))

            if experiment['count_slots'] > len(available_slots):
                print(f'ERROR: experiment_id="{experiment["experiment_id"]}" not enough available slots.')
                unassigned_experiments.append(experiment)
                continue

            # shuffle - чтобы внести случайность, иначе они все упорядочены будут по номеру slot
            # np.random.shuffle(available_slots)
            #         print(f'available_slots: {available_slots}')
            #         print(f'self.slot_to_experiments: {self.slot_to_experiments}')
            available_slots_orderby_count_experiment = sorted(
                available_slots,
                key=lambda x: len(self.slot_to_experiments[x]), reverse=True
            )
            #         print(f'available_slots_orderby_count_experiment: {available_slots_orderby_count_experiment}')

            experiment_slots = available_slots_orderby_count_experiment[:experiment['count_slots']]
            self.experiment_to_slots[experiment['experiment_id']] = experiment_slots
            for slot in experiment_slots:
                self.slot_to_experiments[slot].append(experiment['experiment_id'])
        return unassigned_experiments

    
    def process_user(self, user_id: str):
        """Определяет в какие эксперименты попадает пользователь.

        Сначала нужно определить слот пользователя.
        Затем для каждого эксперимента в этом слоте выбрать пилотную или контрольную группу.

        user_id - идентификатор пользователя.

        return - (int, List[tuple]), слот и список пар (experiment_id, pilot/control group).
            Example: (2, [('exp 3', 'pilot'), ('exp 5', 'control')]).
        """
        # YOUR_CODE_HERE
        hash_value = int(hashlib.md5(str.encode(user_id + str(self.salt_one))).hexdigest(), 16)
        slot = hash_value % self.count_slots
        assignments = []
        for experiment_id in self.slot_to_experiments[slot]:
            for experiment in self.experiments:
                if experiment_id == experiment['experiment_id']:
                    combined_id = str(user_id) + str(experiment_id) + str(self.salt_two)
                    hash_value = int(hashlib.md5(str.encode(combined_id)).hexdigest(), 16)
                    is_pilot = hash_value % 2
                    assignments.append((experiment_id, 'pilot' if is_pilot else 'control'))
        return (slot, assignments)

In [68]:
splitter = ABSplitter(4, "A", "B")

In [69]:
# две группы пилотов, которые не должны пересекаться
group_one = ['pilot 1', 'pilot 2']#, 'pilot 3']
group_two = ['pilot 4', 'pilot 5']

experiments = [
    {'experiment_id': 'pilot 1', 'count_slots': 2, 'conflict_experiments': group_two},
    {'experiment_id': 'pilot 2', 'count_slots': 2, 'conflict_experiments': group_two},
    {'experiment_id': 'pilot 3', 'count_slots': 2, 'conflict_experiments': group_two},
    {'experiment_id': 'pilot 4', 'count_slots': 2, 'conflict_experiments': group_one},
    {'experiment_id': 'pilot 5', 'count_slots': 2, 'conflict_experiments': group_one},
]


unassigned_experiments = splitter.split_experiments(experiments)
print(unassigned_experiments)

[]


In [70]:
splitter.slot_to_experiments[0]

['pilot 1', 'pilot 2', 'pilot 3']

In [71]:
q = splitter.process_user('abc')

In [72]:
q

(2, [('pilot 4', 'pilot'), ('pilot 5', 'control')])

In [23]:
unassigned_experiments

[{'experiment_id': 'pilot 1',
  'count_slots': 2,
  'conflict_experiments': ['pilot 4', 'pilot 5']},
 {'experiment_id': 'pilot 2',
  'count_slots': 2,
  'conflict_experiments': ['pilot 4', 'pilot 5']},
 {'experiment_id': 'pilot 3',
  'count_slots': 2,
  'conflict_experiments': ['pilot 4', 'pilot 5']}]

In [ ]:
experiments = [{'experiment_id': 'exp1'
               ,'count_slots': 3,
               ,'conflict_experiments': ['exp13, exp2']}]

In [8]:
def get_hash_modulo(value: str, modulo: int, salt: str = '0'):
    """Вычисляем остаток от деления: (hash(value) + salt) % modulo."""
    hash_value = int(hashlib.md5(str.encode(str(value) + str(salt))).hexdigest(), 16)
    return hash_value % modulo

def match_pilot_slot_four(experiments: list, slots: list):
    """Функция распределяет пилоты по слотам.

    experiments: список словарей, описывающих пилот. Содержит ключи:
        pilot_id - идентификатор пилота,
        count_slots - необходимое кол-во слотов,
        conflict_experiments - list, идентификаторы несовместных пилотов.
    slots: список с идентификаторами слотов.

    return: словарь соответствия на каких слотах какие пилоты запускаются,
        {slot_id: list_pilot_id, ...}
    """
    experiments = sorted(experiments, key=lambda x: len(x['conflict_experiments']), reverse=True)

    self.slot_to_experiment = {slot: [] for slot in slots}
    self.experiment_to_slots = {experiment['experiment_id']: [] for experiment in experiments}
    unassigned_experiments = []
    for experiment in experiments:
        if experiment['count_slots'] > len(slots):
            print(f'ERROR: experiment_id={experiment["experiment_id"]} needs too many slots.')
            unassigned_experiments.append(experiment)
            continue

        # найдём доступные слоты
        notavailable_slots = []
        for conflict_experiment_id in experiment['conflict_experiments']:
            notavailable_slots += self.experiment_to_slots[conflict_experiment_id]
        available_slots = list(set(slots) - set(notavailable_slots))

        if experiment['count_slots'] > len(available_slots):
            print(f'ERROR: experiment_id="{experiment["experiment_id"]}" not enough available slots.')
            unassigned_experiments.append(experiment)
            continue

        # shuffle - чтобы внести случайность, иначе они все упорядочены будут по номеру slot
        np.random.shuffle(available_slots)
#         print(f'available_slots: {available_slots}')
#         print(f'self.slot_to_experiment: {self.slot_to_experiment}')
        available_slots_orderby_count_experiment = sorted(
            available_slots,
            key=lambda x: len(slot_to_experiment[x]), reverse=True
        )
#         print(f'available_slots_orderby_count_experiment: {available_slots_orderby_count_experiment}')
        
        experiment_slots = available_slots_orderby_count_experiment[:experiment['count_slots']]
        self.experiment_to_slots[experiment['experiment_id']] = experiment_slots
        for slot in experiment_slots:
            self.slot_to_experiment[slot].append(experiment['experiment_id'])
        print()
    return unassigned_experiments


# две группы пилотов, которые не должны пересекаться
group_one = ['pilot 1', 'pilot 2', 'pilot 3']
group_two = ['pilot 4', 'pilot 5']

experiments = [
    {'experiment_id': 'pilot 1', 'count_slots': 2, 'conflict_experiments': group_two},
    {'experiment_id': 'pilot 2', 'count_slots': 2, 'conflict_experiments': group_two},
    {'experiment_id': 'pilot 3', 'count_slots': 2, 'conflict_experiments': group_two},
    {'experiment_id': 'pilot 4', 'count_slots': 2, 'conflict_experiments': group_one},
    {'experiment_id': 'pilot 5', 'count_slots': 2, 'conflict_experiments': group_one},
]

slots = [i for i in range(1, 5)]

match_pilot_slot_four(experiments, slots)

available_slots: [4, 3, 2, 1]
slot_to_experiment: {1: [], 2: [], 3: [], 4: []}
available_slots_orderby_count_experiment: [4, 3, 2, 1]

available_slots: [1, 2, 4, 3]
slot_to_experiment: {1: [], 2: [], 3: ['pilot 4'], 4: ['pilot 4']}
available_slots_orderby_count_experiment: [4, 3, 1, 2]

available_slots: [2, 1]
slot_to_experiment: {1: [], 2: [], 3: ['pilot 4', 'pilot 5'], 4: ['pilot 4', 'pilot 5']}
available_slots_orderby_count_experiment: [2, 1]

available_slots: [2, 1]
slot_to_experiment: {1: ['pilot 1'], 2: ['pilot 1'], 3: ['pilot 4', 'pilot 5'], 4: ['pilot 4', 'pilot 5']}
available_slots_orderby_count_experiment: [2, 1]

available_slots: [2, 1]
slot_to_experiment: {1: ['pilot 1', 'pilot 2'], 2: ['pilot 1', 'pilot 2'], 3: ['pilot 4', 'pilot 5'], 4: ['pilot 4', 'pilot 5']}
available_slots_orderby_count_experiment: [2, 1]



{1: ['pilot 1', 'pilot 2', 'pilot 3'],
 2: ['pilot 1', 'pilot 2', 'pilot 3'],
 3: ['pilot 4', 'pilot 5'],
 4: ['pilot 4', 'pilot 5']}

In [2]:
group_one = ['pilot 1', 'pilot 2', 'pilot 3']
group_two = ['pilot 4', 'pilot 5']

experiments = [
    {'pilot_id': 'pilot 1', 'count_slots': 2, 'conflict_experiments': group_two},
    {'pilot_id': 'pilot 2', 'count_slots': 2, 'conflict_experiments': group_two},
    {'pilot_id': 'pilot 3', 'count_slots': 2, 'conflict_experiments': group_two},
    {'pilot_id': 'pilot 4', 'count_slots': 2, 'conflict_experiments': group_one},
    {'pilot_id': 'pilot 5', 'count_slots': 2, 'conflict_experiments': group_one},
]

experiments = sorted(experiments, key=lambda x: len(x['conflict_experiments']), reverse=True)
experiments

[{'pilot_id': 'pilot 4',
  'count_slots': 2,
  'conflict_experiments': ['pilot 1', 'pilot 2', 'pilot 3']},
 {'pilot_id': 'pilot 5',
  'count_slots': 2,
  'conflict_experiments': ['pilot 1', 'pilot 2', 'pilot 3']},
 {'pilot_id': 'pilot 1',
  'count_slots': 2,
  'conflict_experiments': ['pilot 4', 'pilot 5']},
 {'pilot_id': 'pilot 2',
  'count_slots': 2,
  'conflict_experiments': ['pilot 4', 'pilot 5']},
 {'pilot_id': 'pilot 3',
  'count_slots': 2,
  'conflict_experiments': ['pilot 4', 'pilot 5']}]